In [ ]:
import pandas as pd
import os
import numpy as np
from joblib import Parallel, delayed
from functools import reduce
import math as mt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
colname=['qseqid','sseqid','pident','length','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']

In [ ]:
phagenum=pd.read_table('phagenumprot.txt', header=None, dtype={0:str, 1:str,2:int})
phagenum.columns=['scode','name','numprot']

In [ ]:
phagenum2=phagenum[['scode','name']]
phagenum2.columns=['phage','name']

In [ ]:
for i in range(0,4):
    alpha=0.5+0.1*i
    total={}
    print(alpha)
    for i in os.listdir():
        if i[0:5]=='bestm':
            recop={}
            a=pd.read_table(i, header=None)  #abrir archivo de resultados
            a.columns=colname  #asignar columnas tipo blast
            b=a.groupby('qseqid').head(1)  #top1
            b['scode']=a['sseqid'].apply(lambda x: x.split('.')[0][4:]+'.'+x.split('.')[1]) #transformar codigo
            b['numprot']=a['sseqid'].apply(lambda x: x.split('.')[-1]) #numero de proteinas
            c=b[['scode','numprot']].sort_values(['scode','numprot']) #solo hits
            d=c.groupby(['scode','numprot']).size() #cuenta read por cada proteina
            
            for codigos in pd.unique(d.index.get_level_values(0)):
                recop[codigos]=0
                tmp=d[codigos].values
                while sum(tmp)!=0:
                    valores=np.heaviside(tmp,0)
                    if sum(valores)>=phagenum.loc[phagenum['scode']==codigos,'numprot'].values[0]*alpha:
                        recop[codigos]+=1
                    else:
                        break
                    tmp=tmp-1
                    tmp[tmp<0]=0
            total[i[4:]]=recop
    q=reduce(lambda x,y:pd.merge(x,y,how='outer',on='phage') ,[pd.DataFrame({'phage':list(y.keys()),x:list(y.values())}) for x,y in total.items()])
    qt=q.fillna(0).set_index('phage')
    qt2=qt.loc[:,qt.sum(axis=0)>0]
    gt=pd.merge(qt2.reset_index(),phagenum2,how='left',on='phage')
    gt2=gt.drop('phage',axis=1).set_index('name')
    up=gt2.loc[gt2.sum(axis=1)!=0]
    #up.to_csv("matrixphagesalpha"+str(alpha)+"newmethodestuarinephantome.csv")
    xy=gt2.astype(bool).sum(axis=1)
    xy2=xy[xy!=0]
    xy3=pd.DataFrame({'prev':xy2}).reset_index()
    i=gt2.sum(axis=1).sort_values(ascending=False)
    i2=i[i!=0]
    i3=pd.DataFrame({'detec':i2}).reset_index()
    u=pd.merge(xy3,i3,how='outer',on='name').sort_values('detec',ascending=False)
    #u.to_csv("phagesalpha"+str(alpha)+"newmethodestuarinephantome.csv",index=False)